# YOLOv1

<img src = "https://github.com/fned49/blog/blob/main/YOLO/YOLO-Loss/figure/figure1.png?raw=true">

기존 R-CNN 계열의 모델이 classification, localization task에 맞게 서로 다른 loss function을 사용했던 것과 달리,  
YOLO v1 모델은 regression 시 주로 사용되는 SSE(Sum of Squared Error)를 사용합니다.  
위의 그림에서 볼 수 있듯이 Localization loss, Confidence loss, Classification loss의 합으로 구성되어 있습니다. 

#### Localization loss

$$\lambda_{coord}\sum_{i=0}^{S^2}\sum_{j=0}^{B}\mathcal{1}_{ij}^{obj}\left[\left(x_i-\hat{x}_i\right)^2 + \left(y_i-\hat{y}_i\right)^2\right]$$  
$$+\lambda_{coord}\sum_{i=0}^{S^2}\sum_{j=0}^{B}\mathcal{1}_{ij}^{obj}\left[\left(\sqrt{w_i}-\sqrt{\hat{w}_i}\right)^2 + \left(\sqrt{h_i}-\sqrt{\hat{h}_i}\right)^2\right]$$ 

- $\lambda_{coord}$ : 대부분의 grid cell은 객체를 포함하지 않아 confidence score가 0이 되어, 객체를 포함하는 grid cell의 gradient를 압도하므로 모델이 불안정해질 수 있습니다. $\lambda_{coord}$는 이러한 문제를 해결하기 위해 객체를 포함하는 cell에 가중치를 두는 파라미터입니다. 논문에서는 $\lambda_{coord} = 5$로 설정합니다.
- $S^2$ : grid cell의 수(=7x7=49)
- $B$ : 각 grid cell의 bounding box의 수(=2)
- $\mathcal{1}_{ij}^{obj}$ : $i$번째 grid cell의 $j$번째 bounding box가 객체를 예측하도록 할당 받았을 때 1, 그렇지 않을 경우 0. grid cell에서는 B개의 bounding box를 예측하지만, 그 중 confidence score가 가장 높은 1개의 bounding box만을 학습에 사용합니다.
- $x_i$, $y_i$, $w_i$, $h_i$ : ground truth box의 $x$, $y$ 좌표와 width, height 크기가 큰 bounding box의 small error가 크기가 작은 bounding box의 error보다 덜 중요하다는 것을 반영하기 위해 $w_i$, $h_i$ 값에 squre root를 씌어주게 됩니다. 
- $\hat{x}_i$, $\hat{y}_i$, $\hat{w}_i$, $\hat{h}_i$ : 예측 bounding box의 $x, y$ 좌표, width, height

#### Confidence loss

$$+\sum_{i=0}^{S^2}\sum_{j=0}^{B}\mathcal{1}_{ij}^{obj}\left(C_i-\hat{C}_i\right)^2$$  
$$+\lambda_{noobj}\sum_{i=0}^{S^2}\sum_{j=0}^{B}\mathcal{1}_{ij}^{noobj}\left(C_i-\hat{C}_i\right)^2$$  

- $\lambda_{noobj}$ : 앞서 언급한 객체를 포함하지 않는 grid cell에 곱해주는 가중치 파라미터입니다. 논문에서는 $\lambda_{noobj}=0.5$로 설정했습니다.\
$\lambda_{obj}$로 설정한 것에 비해 상당히 작게 설정하여 객체를 포함하지 않은 grid cell의 영향력을 줄였습니다. 
- $\mathcal{1}_{ij}^{noobj}$ : $i$번째 grid cell의 $j$번째 bounding box가 객체를 예측하도록 할당(responsible)받지 않았을 때 1, 그렇지 않을 경우 0
- $C_i$ : 객체가 포함되어 있을 경우 1, 그렇지 않을 경우 0
- $\hat{C}_i$ : 예측한 bounding box의 confidence score


#### Classification loss

$$+\sum_{i=0}^{S^2}\mathcal{1}_{i}^{obj}\sum_{c\in classes}\left(p_i\left(c\right)-\hat{p}_i\left(c\right)\right)$$  

- $p_i\left(c\right)$ : 실제 class probabilities
- $\hat{p}_i\left(c\right)$ : 예측한 class probabilities

---

# YOLOv2

<img src = "https://github.com/fned49/blog/blob/main/YOLO/YOLO-Loss/figure/figure2.png?raw=true">

YOLO v1과 같이 Localization loss, Confidence loss, Classification loss로 구성되어 있고,  
또한 v1과 같이 전체 loss가 SSE(Sum of Squared Error)입니다.

$$\lambda^{coord}_{obj}\sum_{i}^{S^2}\sum_{j}^{B}\mathcal{1}_{ij}^{responsible\_obj}\left[\left(x_{ij}^{pred}-\hat{x}_{ij}^{obj}\right)^2 + \left(y_i-\hat{y}_i\right)^2\right]$$  
$$+\lambda_{coord}\sum_{i=0}^{S^2}\sum_{j=0}^{B}\mathcal{1}_{ij}^{obj}\left[\left(\sqrt{w_i}-\sqrt{\hat{w}_i}\right)^2 + \left(\sqrt{h_i}-\sqrt{\hat{h}_i}\right)^2\right]$$ 